In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Mofid

/content/drive/MyDrive/Mofid


In [ ]:
!unzip train-2.zip

Archive:  train-2.zip
   creating: train/
  inflating: train/train.csv         
  inflating: train/test.csv          


In [ ]:
%cd train-2

/content/drive/My Drive/Mofid/train-2


In [ ]:
!pip install catboost

     |████████████████████████████████| 65.8MB 45kB/s 


In [ ]:
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
df_train = pd.read_csv('train.csv')
df_train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0.0,1.0,0.0,F,N,Blue,Circle,Hamster,Costa Rica,Theremin,b165c9589,6fc4f10fb,22a1b223f,1a59581be,e184f0df9,2.0,Master,Freezing,e,N,oh,3.0,5.0,0
1,1.0,1.0,0.0,F,Y,Blue,Trapezoid,Dog,India,Theremin,0289ab250,c3b174e52,5e17197af,ca9ad1d4b,a91d577f9,1.0,Contributor,Freezing,f,N,Kq,3.0,8.0,0
2,0.0,0.0,0.0,F,N,Red,Circle,Hamster,India,Bassoon,337a3bd19,c18193dac,56d35c774,32ff72b97,7813e67c3,2.0,NaN,Cold,i,D,Fl,3.0,3.0,0
3,0.0,0.0,0.0,T,N,Green,Polygon,Hamster,India,Theremin,024efa364,9043b0882,e60e65034,46a005ddf,55fe9a4b8,1.0,Grandmaster,Cold,n,Z,In,7.0,7.0,0
4,0.0,0.0,0.0,F,N,Blue,Polygon,Axolotl,Costa Rica,Bassoon,7b0da9c0f,7a22f53ea,f74f0b894,NaN,512255ae7,1.0,Expert,Hot,i,X,Sk,5.0,2.0,1


In [ ]:
df_train.drop(['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9'], inplace = True, axis = 1) 
df_train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0.0,1.0,0.0,F,N,Blue,Circle,Hamster,Costa Rica,Theremin,2.0,Master,Freezing,e,N,oh,3.0,5.0,0
1,1.0,1.0,0.0,F,Y,Blue,Trapezoid,Dog,India,Theremin,1.0,Contributor,Freezing,f,N,Kq,3.0,8.0,0
2,0.0,0.0,0.0,F,N,Red,Circle,Hamster,India,Bassoon,2.0,NaN,Cold,i,D,Fl,3.0,3.0,0
3,0.0,0.0,0.0,T,N,Green,Polygon,Hamster,India,Theremin,1.0,Grandmaster,Cold,n,Z,In,7.0,7.0,0
4,0.0,0.0,0.0,F,N,Blue,Polygon,Axolotl,Costa Rica,Bassoon,1.0,Expert,Hot,i,X,Sk,5.0,2.0,1


In [ ]:
df_train.shape

(480000, 19)

In [ ]:
df_train.isna().sum()

bin_0     14327
bin_1     14383
bin_2     14383
bin_3     14554
bin_4     14449
nom_0     14633
nom_1     14531
nom_2     14441
nom_3     14424
nom_4     14373
ord_0     14707
ord_1     14402
ord_2     14448
ord_3     14263
ord_4     14443
ord_5     14195
day       14310
month     14428
target        0
dtype: int64

In [ ]:
df_train.dtypes

bin_0     float64
bin_1     float64
bin_2     float64
bin_3      object
bin_4      object
nom_0      object
nom_1      object
nom_2      object
nom_3      object
nom_4      object
ord_0     float64
ord_1      object
ord_2      object
ord_3      object
ord_4      object
ord_5      object
day       float64
month     float64
target      int64
dtype: object

In [ ]:
for column in df_train.columns:
    df_train[column].fillna(df_train[column].mode()[0], inplace=True)

In [ ]:
df_train.isna().sum()

bin_0     0
bin_1     0
bin_2     0
bin_3     0
bin_4     0
nom_0     0
nom_1     0
nom_2     0
nom_3     0
nom_4     0
ord_0     0
ord_1     0
ord_2     0
ord_3     0
ord_4     0
ord_5     0
day       0
month     0
target    0
dtype: int64

***
Validation

In [ ]:
X = df_train.loc[:, df_train.columns != 'target']
y = df_train['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    stratify=y, 
                                                    test_size=0.3)

***
Modeling

In [ ]:
cat_features = df_train[['bin_3', 'bin_4', 'nom_0', 'nom_1', 'nom_2', 'nom_3', 
                         'nom_4', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']]

In [ ]:
train_dataset = Pool(data=X_train,
                     label=y_train,
                     cat_features=cat_features)

In [ ]:
eval_dataset = Pool(data=X_test,
                    label=y_test,
                    cat_features=cat_features)

In [ ]:
model = CatBoostClassifier(
    iterations=500, 
    #task_type="GPU",
    #devices='0:1',
    early_stopping_rounds = 5,
    custom_metric=['AUC'])

In [ ]:
model.fit(train_dataset,
          eval_set=eval_dataset)

Learning rate set to 0.180245
0:	learn: 0.6170795	test: 0.6170152	best: 0.6170152 (0)	total: 355ms	remaining: 2m 57s
1:	learn: 0.5552384	test: 0.5548388	best: 0.5548388 (1)	total: 1.1s	remaining: 4m 34s
2:	learn: 0.5166028	test: 0.5165161	best: 0.5165161 (2)	total: 1.64s	remaining: 4m 31s
3:	learn: 0.4892186	test: 0.4892427	best: 0.4892427 (3)	total: 2.34s	remaining: 4m 49s
4:	learn: 0.4712964	test: 0.4710634	best: 0.4710634 (4)	total: 3.07s	remaining: 5m 4s
5:	learn: 0.4599965	test: 0.4596160	best: 0.4596160 (5)	total: 3.74s	remaining: 5m 7s
6:	learn: 0.4519152	test: 0.4514952	best: 0.4514952 (6)	total: 4.55s	remaining: 5m 20s
7:	learn: 0.4462190	test: 0.4457019	best: 0.4457019 (7)	total: 5.25s	remaining: 5m 23s
8:	learn: 0.4421118	test: 0.4416323	best: 0.4416323 (8)	total: 6.01s	remaining: 5m 28s
9:	learn: 0.4388522	test: 0.4382716	best: 0.4382716 (9)	total: 6.8s	remaining: 5m 33s
10:	learn: 0.4365832	test: 0.4359103	best: 0.4359103 (10)	total: 7.43s	remaining: 5m 30s
11:	learn: 0.43

***
Test

In [ ]:
df_test = pd.read_csv('test.csv')
df_test.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,1.0,1.0,0.0,F,N,Red,Triangle,Axolotl,Russia,Theremin,ed6509b47,8e8eaadde,01103c3e4,8d13bd3e9,65b262989,2.0,Master,Warm,a,Y,MV,7.0,5.0
1,0.0,0.0,0.0,F,Y,Blue,Triangle,Hamster,Costa Rica,Theremin,NaN,1ab15830f,adbdefba3,7861df0a8,31dce772f,1.0,Grandmaster,Freezing,c,H,GZ,4.0,3.0
2,1.0,0.0,0.0,F,N,Blue,Circle,Axolotl,Russia,Theremin,940e1a8f0,3f229cb82,9d4a94d03,cccbca824,c9a07c607,3.0,Expert,Cold,f,F,gc,NaN,6.0
3,0.0,0.0,NaN,F,N,Blue,Polygon,Axolotl,Costa Rica,Bassoon,ba688b662,89545cc37,e06abf51f,154aacc1c,eee9ce48c,2.0,Expert,Warm,f,Y,NaN,6.0,1.0
4,0.0,0.0,0.0,T,N,Red,Polygon,Lion,Russia,Bassoon,9e7d06868,b5a223cde,018382adf,1fe15a999,0a55dd91d,3.0,NaN,Hot,c,K,Hk,1.0,5.0


In [ ]:
df_test.drop(['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9'], inplace = True, axis = 1) 

In [ ]:
df_test.isna().sum()

bin_0    3567
bin_1    3620
bin_2    3547
bin_3    3460
bin_4    3598
nom_0    3619
nom_1    3625
nom_2    3594
nom_3    3697
nom_4    3662
ord_0    3581
ord_1    3639
ord_2    3627
ord_3    3653
ord_4    3487
ord_5    3518
day      3642
month    3560
dtype: int64

In [ ]:
for column in df_test.columns:
    df_test[column].fillna(df_test[column].mode()[0], inplace=True)

In [ ]:
df_test.isna().sum()

bin_0    0
bin_1    0
bin_2    0
bin_3    0
bin_4    0
nom_0    0
nom_1    0
nom_2    0
nom_3    0
nom_4    0
ord_0    0
ord_1    0
ord_2    0
ord_3    0
ord_4    0
ord_5    0
day      0
month    0
dtype: int64

In [ ]:
test_dataset = Pool(data=df_test,
                    cat_features=cat_features)

In [ ]:
preds_class = model.predict_proba(test_dataset)
preds_class

array([[0.95825592, 0.04174408],
       [0.98531333, 0.01468667],
       [0.92350955, 0.07649045],
       ...,
       [0.81570859, 0.18429141],
       [0.86085624, 0.13914376],
       [0.97649872, 0.02350128]])

In [ ]:
out = pd.DataFrame(preds_class)
out

,0,1
0,0.958256,0.041744
1,0.985313,0.014687
2,0.923510,0.076490
3,0.848226,0.151774
4,0.840345,0.159655
...,...,...
119995,0.368848,0.631152
119996,0.429761,0.570239
119997,0.815709,0.184291
119998,0.860856,0.139144


In [ ]:
out[1].to_csv('submission.csv', index=False, header=['target'])